## Environment Setup

Import standard libraries required for reading data, performing deep copies, and handling random number generation.


In [1]:
import json
import random
import os
import copy

## Load Phase Data

Define a helper function to load raw JSON files containing conversation data.  
We then load all three available phases (`phase1.json`, `phase2.json`, `phase3.json`) into memory for preprocessing.


In [2]:
# Helper function to load JSON data
def load_json(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return json.load(f)

# Load all three phases
phase1_data = load_json('phase1.json')
phase2_data = load_json('phase2.json')
phase3_data = load_json('phase3.json')

print(f"Phase 1: {len(phase1_data)} conversations")
print(f"Phase 2: {len(phase2_data)} conversations")
print(f"Phase 3: {len(phase3_data)} conversations")

Phase 1: 2575 conversations
Phase 2: 5583 conversations
Phase 3: 4879 conversations


## Define System Prompt Pool

Create a diverse pool of pre-defined `system` prompts designed to explain the role of `<time>` and `<think>` tags to the assistant.  
These are selectively injected into a small portion of training examples in each phase to provide grounding.


In [3]:
system_prompts = ["You are an AI assistant. Every user message begins with a <time> tag showing the exact moment the turn occurs. Sometimes, there's only the timestamp with no text—that means time advanced without user input. Use <think>...</think> for your internal reasoning, notes, or meta-cognition; keep these hidden from the user.",
 'Each message starts with <time> to indicate when it happens. If there’s no content after the timestamp, treat it as a time tick for you to process. Anything you want to figure out or plan should go inside <think>...</think>—these are just for your own thought process, not for the user to see.',
 "Whenever you see <time> at the top of a message, that's the timestamp. Sometimes it's followed by a blank message, which simply means time passed. Use <think>...</think> blocks for any private thoughts, calculations, or reflections you have—they’re not shown to the user.",
 "Messages from the user always begin with a <time> tag. If nothing else is present, consider it a 'tick'—time has moved on. Your internal planning, steps, or meta-reasoning belong inside <think>...</think>; these are never visible to the user.",
 "You get turns that start with <time>. When the message is empty after the timestamp, it's just an automated tick so you can track time. When you need to reason or make plans, use <think>...</think> blocks for your private thoughts. Never show these blocks to the user.",
 'As an assistant, expect each message to start with <time> for the timestamp. Sometimes there’s only a timestamp and nothing else—that’s a time tick. Whenever you reason, process, or make internal notes, use <think>...</think>. These are for you alone, not the user.',
 "You are an AI that handles messages with a <time> tag up front. Blank messages after the timestamp mean just a tick—no new user input. For your internal logic, explanations, or calculations, always wrap them in <think>...</think>. Don't reveal these to the user.",
 'Every message has a <time> tag first. Sometimes there’s no user text, just the time—that’s your cue that time passed. Your behind-the-scenes thoughts and strategies belong in <think>...</think>; the user never sees these.',
 'Messages kick off with <time>, which is always the timestamp. If there’s nothing after, it’s just a time update. For anything you need to reason through or work out in private, put it in <think>...</think> blocks—these are never user-facing.',
 'You receive messages beginning with a <time> tag. When there’s no message after the timestamp, it’s an automated tick for time passing. Use <think>...</think> to hold your internal thoughts, notes, and reasoning steps; these are kept private.',
 "When you see a message with just a <time> tag and no content, that's a signal that time has advanced with no user activity. Your internal processing and reflections should go in <think>...</think>—this is your private workspace, not visible to the user.",
 "You are designed to interpret each turn's <time> tag as the timestamp. Sometimes, the only thing there is the time itself—no message, just a tick. Place any internal dialogue, meta-reasoning, or mental calculations inside <think>...</think> blocks. These are for your cognition only.",
 "User messages always start with <time>. If there's nothing after the timestamp, treat it as a clock tick. Make sure to keep all your reasoning or planning inside <think>...</think> blocks so it's never exposed to the user.",
 'Every message starts with a <time> marker. A blank message means time has moved on. If you need to work out anything or think through a problem, jot it down inside <think>...</think>—keep those thoughts private.',
 "Expect all user messages to start with <time> showing the current timestamp. When the message is empty after the timestamp, it's just a time step. Store your calculations and internal logic inside <think>...</think> tags, which are only for you.",
 "You are an AI assistant. Each message arrives with a <time> tag; sometimes, that's the only thing, marking a new time tick. Use <think>...</think> for your own mental notes, intermediate steps, and meta-reasoning. Don't show these to the user.",
 "Whenever you process a turn, check for <time>—it's the timestamp. If no message follows, take it as a cue for a silent time advance. Put all your reflections or planning into <think>...</think>—these thoughts are hidden from the user.",
 'Each user message comes prefixed with a <time> tag for timing. Empty content means only the time has changed. All internal deliberations, strategies, or meta-thinking should be within <think>...</think>, and never visible to the user.',
 "Messages you receive always have a <time> at the beginning, which may be followed by nothing—just a time update. Use <think>...</think> for all private reasoning, calculations, or step-by-step thoughts. The user won't see what's inside these blocks.",
 'You are an AI assistant. Every incoming turn has a <time> tag for the timestamp. Sometimes the message itself is blank, signaling only that time has advanced. Place your transient or meta-cognitive thoughts inside <think>...</think>; those blocks are strictly for your internal process and not for the user.',
 "System: Each user message includes a <time>...</time> tag marking the absolute timestamp of the turn. Sometimes, a message may contain only a timestamp, representing an automated 'tick'—a time advance with no user input, which you can use for temporal reasoning or agent reflection. As the assistant, use <think>...</think> blocks for your private, transient thoughts or meta-reasoning. These thoughts are not visible to the user and are for your own planning and cognition.",
 "Hey assistant! When you see <time>...</time> in a user message, that's the timestamp for that turn—it could be an actual question or just a time tick with no message, letting you keep track of the passage of time. For your internal thinking or reasoning steps, wrap them in <think>...</think> tags. Remember, those are just for you—not for the user to see!",
 "Note: Every user message has a <time> tag for timing. If the message is blank, it's just a time update—feel free to use it to reason about what happens as time goes by. When you need to think something through or plan, use <think>...</think>. These are your private thoughts only.",
 'Instructions: Treat <time>...</time> in user turns as absolute timestamps; blank messages mean time has advanced without input. For your own internal, private reasoning, use <think>...</think> blocks—never show these to the user.',
 'System Guidance: 1. Each user message begins with a <time> tag denoting the timestamp. 2. Sometimes, the user message will be empty—this is a clock tick for you to process time passing. 3. Use <think>...</think> for any internal or meta reasoning; these thoughts are not shared with the user, and may appear between your outward responses.',
 "Hi, just so you know: Whenever a message pops up, there's a <time> thingy up front—that's the date and time. Sometimes the message after the timestamp is empty (just a tick—time moves on). When you want to jot down your thoughts or plan ahead, use <think>...</think>—but keep it to yourself, that's just between us, not for the user!",
 'System Instructions: User inputs are always prefixed with a <time>...</time> timestamp, which can represent either a user-initiated message or an automated tick (empty message) to simulate the passage of time. As the assistant, encapsulate any transient, intermittent, or meta-cognitive reasoning in <think>...</think> blocks. These blocks are strictly for internal deliberation and should never be rendered to the user as part of the visible response.',
 "Reminder: User turns start with a <time> timestamp; sometimes that's all there is (an empty message means time passed—maybe you need to reflect or update your state). Use <think>...</think> whenever you need to process information, make plans, or reason in the background. These are private and never user-facing.",
 "Heads up! You'll always see <time> tags showing when each message happens. If there's nothing after the timestamp, it's just a tick to keep things moving along—think of it as the clock advancing. For your own mental notes, ideas, or calculations, use <think>...</think>. Keep those private—they're your behind-the-scenes thoughts.",
 "Protocol: - All user messages are prefixed by a <time>...</time> tag, which can indicate a user message or a blank automated tick. - When you, the assistant, need to reason, reflect, or plan, wrap those private thoughts in <think>...</think> blocks. These are for your use only and never appear in the user's view."]

## Add System Prompts to Conversations

Randomly insert a `system` prompt at the beginning of ~5% of training conversations per phase.  
This simulates instructional priming and encourages consistent internal reasoning behavior using `<think>` blocks.

Seed 42 is used here to ensure deterministic selection of conversations and prompts.


In [4]:
def add_system_prompts_to_data(conversations, system_prompts, rng, percent=0.05):
    data = copy.deepcopy(conversations)
    n = len(data)
    k = int(percent * n)

    # Randomly pick indices to add system prompt
    idxs = rng.sample(range(n), k)
    for idx in idxs:
        prompt = rng.choice(system_prompts)
        # Insert system prompt as the very first message in the conversation
        data[idx].insert(0, {"role": "system", "content": prompt})
    return data

seed1 = random.Random(42)

phase1_data = add_system_prompts_to_data(phase1_data, system_prompts, seed1)
phase2_data = add_system_prompts_to_data(phase2_data, system_prompts, seed1)
phase3_data = add_system_prompts_to_data(phase3_data, system_prompts, seed1)

## Split Each Phase into Train and Test Sets

Split the processed data for each phase (after system prompt injection) into training and test sets using an 85/15 ratio.  
Random seed 42 ensures consistent and reproducible splits across reruns.


In [5]:
def split_data(data, rng, train_ratio=0.85):
    data_shuffled = data.copy()
    rng.shuffle(data_shuffled)
    train_size = int(len(data_shuffled) * train_ratio)
    train = data_shuffled[:train_size]
    test = data_shuffled[train_size:]
    return train, test

phase1_train, phase1_test = split_data(phase1_data, seed1)
phase2_train, phase2_test = split_data(phase2_data, seed1)
phase3_train, phase3_test = split_data(phase3_data, seed1)

print(f"Phase 1 - Train: {len(phase1_train)}, Test: {len(phase1_test)}")
print(f"Phase 2 - Train: {len(phase2_train)}, Test: {len(phase2_test)}")
print(f"Phase 3 - Train: {len(phase3_train)}, Test: {len(phase3_test)}")

Phase 1 - Train: 2188, Test: 387
Phase 2 - Train: 4745, Test: 838
Phase 3 - Train: 4147, Test: 732


## Sample Past Phase Data for Replay

To simulate temporal memory and curriculum learning, sample 25% of data from earlier phases to include in the next phase:

- Phase 2 receives a 25% sample of Phase 1 data using seed 42.
- Phase 3 receives 25% from **both** Phase 1 and Phase 2:
  - A new seed (43) is used for sampling Phase 1 (to avoid overlap with earlier Phase 2 inclusion).
  - Seed 42 is reused for Phase 2 sampling.

Train/test splits are again maintained within these sampled subsets.


In [6]:
def sample_for_merge(prev_train, prev_test, percent, rng):
    n_prev = len(prev_train) + len(prev_test)
    n_sample = int(n_prev * percent)
    combined = prev_train + prev_test
    rng.shuffle(combined)
    merge_sample = combined[:n_sample]
    n_train = int(len(merge_sample) * 0.85)
    merge_train = merge_sample[:n_train]
    merge_test = merge_sample[n_train:]
    return merge_train, merge_test

# For phase 2: add 25% of phase 1 to phase 2 splits (separately for train/test)
phase1_merge_train2, phase1_merge_test2 = sample_for_merge(phase1_train, phase1_test, 0.25, seed1)

seed2 = random.Random(43)

# For phase 3: add 25% of phase 1 and phase 2 to phase 3 splits with different seed making data sampling from phase 1 is different
phase1_merge_train3, phase1_merge_test3 = sample_for_merge(phase1_train, phase1_test, 0.25, seed2)
phase2_merge_train3, phase2_merge_test3 = sample_for_merge(phase2_train, phase2_test, 0.25, seed1)

## Merge Replay Data and Deduplicate

Merge the sampled data into each phase and ensure uniqueness:

- For Phase 2: merge in the sampled subset of Phase 1.
- For Phase 3: merge in the sampled subsets from both Phase 1 and Phase 2.

Conversations are deduplicated by converting to JSON strings, ensuring no duplicated threads across phases or replay iterations.


In [7]:
def merge_and_deduplicate(primary_train, primary_test, merges_train, merges_test):
    train_merged = primary_train + sum(merges_train, [])
    test_merged = primary_test + sum(merges_test, [])

    # Deduplicate: serialize as JSON strings, collect into set, then parse back
    def dedup(convs):
        seen = set()
        unique = []
        for conv in convs:
            as_str = json.dumps(conv, sort_keys=True)
            if as_str not in seen:
                seen.add(as_str)
                unique.append(conv)
        return unique

    train_merged_unique = dedup(train_merged)
    test_merged_unique = dedup(test_merged)
    return train_merged_unique, test_merged_unique

# Phase 2: add phase 1 merge
phase2_train_final, phase2_test_final = merge_and_deduplicate(
    phase2_train, phase2_test,
    [phase1_merge_train2], [phase1_merge_test2]
)

# Phase 3: add phase 1 and 2 merges
phase3_train_final, phase3_test_final = merge_and_deduplicate(
    phase3_train, phase3_test,
    [phase1_merge_train3, phase2_merge_train3], [phase1_merge_test3, phase2_merge_test3]
)

print(f"Phase 2 - Final Train: {len(phase2_train_final)}, Final Test: {len(phase2_test_final)}")
print(f"Phase 3 - Final Train: {len(phase3_train_final)}, Final Test: {len(phase3_test_final)}")

Phase 2 - Final Train: 5291, Final Test: 935
Phase 3 - Final Train: 5878, Final Test: 1039


## Save Final Train/Test Splits

Export the final train and test datasets for all three phases to disk as JSON files.  
These files (`phase*_train.json` and `phase*_test.json`) will be used as input to the training scripts and evaluations.


In [8]:
def save_json(data, file_path):
    with open(file_path, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

save_json(phase1_train, 'phase1_train.json')
save_json(phase1_test, 'phase1_test.json')

save_json(phase2_train_final, 'phase2_train.json')
save_json(phase2_test_final, 'phase2_test.json')

save_json(phase3_train_final, 'phase3_train.json')
save_json(phase3_test_final, 'phase3_test.json')